# Importing Libraries

In [ ]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import squarify



# 1.Data Profiling and Quality Assessment, Load and inspect the dataset.

### 1.1 Loading Dataset.

In [ ]:
df = pd.read_csv('global_tech_sales.csv')
df.head()

### 1.2 Identify missing values, duplicates, inconsistent data types, and outliers.

In [ ]:

missing_values = df.isnull().sum()

duplicate_rows = df.duplicated().sum()

data_types = df.dtypes

outliers = {}
for col in ['Sales', 'Profit', 'Discount', 'Quantity']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]

print("Missing Values:\n", missing_values)
print("\nNumber of Duplicate Rows:", duplicate_rows)
print("\nData Types:\n", data_types)
print("\nOutliers:\n", outliers)

# 2.Univariate and Bivariate Analysis.

### 2.1 Univariate Analysis using histograms and boxplots for numerical columns.

In [ ]:
#Profits by Region
profits_per_region = df.groupby('Region')['Profit'].sum().sort_values(ascending=False)
print(profits_per_region)



In [ ]:
#Profits by Category
profits_per_category = df.groupby('Category')['Profit'].sum().sort_values(ascending=False)
print(profits_per_category)

In [ ]:
for col in ['Sales', 'Profit', 'Discount']:
    plt.figure(figsize=(12, 6))
    
    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, bins=30, color='blue')
    plt.title(f'Histogram of {col}')
 

In [ ]:
   
# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df[col], color='orange')
plt.title(f'Boxplot of {col}')
    
plt.tight_layout()
plt.show()

In [ ]:

# Univariate Analysis using countplot for categorical column
plt.figure(figsize=(10, 6))
sns.countplot(y='Category', data=df, order=df['Category'].value_counts().index, palette='viridis')
plt.title('Countplot of Category')
plt.show()

### 2.2 Bivariate Analysis using scatter plots and correlation heatmap.

In [ ]:
# Pairplot for numerical columns
numerical_columns = ['Sales', 'Profit', 'Discount', 'Quantity']
sns.pairplot(df[numerical_columns], diag_kind='kde', corner=True, plot_kws={'alpha': 0.7})
plt.suptitle('Pairplot of Numerical Columns', y=1.02, fontsize=16)
plt.show()

In [ ]:
# Correlation heatmap for numerical columns
plt.figure(figsize=(10, 8))
correlation_matrix = df[numerical_columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Columns', fontsize=16)
plt.show()

In [ ]:
# Highlighting anomalies in scatterplots
plt.figure(figsize=(16, 12))

num_cols = len(numerical_columns)  # Define num_cols as the number of numerical columns
plot_index = 1

# Create a boolean Series for anomalies based on a condition (e.g., outliers in 'Profit')
anomalies = (df['Profit'] < lower_bound) | (df['Profit'] > upper_bound)

for i, col1 in enumerate(numerical_columns):
    for j, col2 in enumerate(numerical_columns):
        if i < j:  # Avoid duplicate plots
            plt.subplot(num_cols - 1, num_cols - 1, plot_index)
            sns.scatterplot(x=df[col1], y=df[col2], alpha=0.7, hue=anomalies, palette={True: 'red', False: 'blue'})
            plt.title(f'{col1} vs {col2}')
            plt.xlabel(col1)
            plt.ylabel(col2)
            plot_index += 1

plt.tight_layout()
plt.show()

In [ ]:
# Sales per category segmented by region
sales_per_category_region = df.groupby(['Region', 'Category'])['Sales'].sum().unstack()

print(sales_per_category_region)

sales_per_category_region.plot(kind='bar', figsize=(12, 8), stacked=True, colormap='viridis', edgecolor='black')
plt.title('Sales per Category Segmented by Region', fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Category', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


highest_sales_per_category = sales_per_category_region.idxmax()
print("Regions with Highest Sales in Each Category:")
print(highest_sales_per_category)

# 3. Time Series and Trend Analysis.

### 3.1 Convert date-related fields and analyze monthly/yearly sales trends.

In [ ]:

df['OrderDate'] = pd.to_datetime(df['OrderDate'])

df['Year'] = df['OrderDate'].dt.year
df['Month'] = df['OrderDate'].dt.month

#Yearly sales trends analysis
yearly_sales = df.groupby('Year')['Sales'].sum()
print("Yearly Sales Trends:\n", yearly_sales)

#Monthly sales trends analysis
monthly_sales = df.groupby(['Year', 'Month'])['Sales'].sum().unstack()
print("\nMonthly Sales Trends:\n", monthly_sales)


In [ ]:
# Yearly sales trends 
plt.figure(figsize=(10, 6))
yearly_sales.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Yearly Sales Trends', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#Monthly sales trends
plt.figure(figsize=(12, 8))
sns.heatmap(segmented_summary, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5)
plt.title('Sales and Profit by Region', fontsize=16)
plt.xlabel('Metrics', fontsize=12)
plt.ylabel('Region', fontsize=12)
plt.yticks(ticks=np.arange(len(segmented_summary.index)) + 0.5, labels=segmented_summary.index, rotation=0)
plt.show()

### 3.2 Time series showing seasonality and growth.

In [ ]:
# Time series showing seasonality and growth
plt.figure(figsize=(14, 8))
monthly_sales.T.plot(figsize=(14, 8), marker='o')
plt.title('Seasonality and Growth in Monthly Sales', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.legend(title='Year', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### 3.3 Identify and explain any unusual dips or spikes in data.

In [ ]:
# Calculating monthly sales change
monthly_sales_change = monthly_sales.diff(axis=1)

# Months with unusual dips or spikes
unusual_changes = monthly_sales_change[(monthly_sales_change < lower_bound) | (monthly_sales_change > upper_bound)]
print("Unusual Dips or Spikes in Data:")
print(unusual_changes)



# 4. Segment-Level Insights.

### 4.1 Segment the data by Region, Customer Segment, and/or Category.

In [ ]:
# Segmenting the data by Region, Customer Segment, and Category
segmented_data = df.groupby(['Region', 'CustomerSegment', 'Category']).agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum',
    'Discount': 'mean'
}).reset_index()

print(segmented_data.head())

### 4.2 Use grouped bar charts or treemaps to visualize performance across segment

In [ ]:
# Grouped bar chart for Sales and Profit by Region
segmented_summary = segmented_data.groupby('Region')[['Sales', 'Profit']].sum()

segmented_summary.plot(kind='bar', figsize=(12, 6), color=['skyblue', 'orange'], edgecolor='black')
plt.title('Sales and Profit by Region', fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Amount', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Metrics', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Define region_sales based on segmented_summary
region_sales = segmented_summary.reset_index()
print(region_sales)
print(profits_per_region)




In [ ]:
print(region_sales)

In [ ]:
# Treemap for Sales by Region
region_sales['adjusted_size'] = region_sales['Sales'] / region_sales['Sales'].sum() * 100

plt.figure(figsize=(12, 8))
squarify.plot(sizes=region_sales['adjusted_size'], label=region_sales['Region'], alpha=0.8, color=sns.color_palette('Set2'))
plt.title('Global Tech Sales by Region (Adjusted Sizes)', fontsize=16)
plt.axis('off')
plt.show()

print(region_sales)

In [ ]:
#Key insights
print("Key Insights:")
print("1. High-performing regions: North America and Europe with profits of", profits_per_region['North America'], "and", profits_per_region['Europe'], "respectively.")
print("2. South America has the highest average discount rate, which might be impacting profit margins.")
print("3. East Asia region shows significantly low sales compared to other regions.")
print("4. Discounts have a negative correlation with profit (-0.143276), indicating their impact on margins.")
print("5. Monthly sales trends reveal seasonal patterns, suggesting opportunities for targeted promotions.")

#  5. Executive Summary and Dashboard Elements

### Final Summary with Key Insights

In [ ]:

print("Final Summary:")
print("1. North America and Europe are the top-performing regions in terms of profit, indicating strong market presence and customer base.")
print("2. South America has the highest average discount rate, which may be negatively impacting profit margins. A review of discount strategies is recommended.")
print("3. Seasonal trends in monthly sales suggest opportunities for targeted marketing campaigns during peak months to maximize revenue.")



### Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Sales by Region (Bar Chart)
region_sales.plot(kind='bar', x='Region', y='Sales', ax=axes[0, 0], color='skyblue', edgecolor='black')
axes[0, 0].set_title('Sales by Region', fontsize=14)
axes[0, 0].set_xlabel('Region', fontsize=12)
axes[0, 0].set_ylabel('Sales', fontsize=12)
axes[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Profit by Region (Bar Chart)
region_sales.plot(kind='bar', x='Region', y='Profit', ax=axes[0, 1], color='orange', edgecolor='black')
axes[0, 1].set_title('Profit by Region', fontsize=14)
axes[0, 1].set_xlabel('Region', fontsize=12)
axes[0, 1].set_ylabel('Profit', fontsize=12)
axes[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Treemap for Sales by Region
squarify.plot(sizes=region_sales['adjusted_size'], label=region_sales['Region'], alpha=0.8, color=sns.color_palette('Set2'), ax=axes[1, 0])
axes[1, 0].set_title('Treemap of Sales by Region', fontsize=14)
axes[1, 0].axis('off')

# Correlation Heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, ax=axes[1, 1])
axes[1, 1].set_title('Correlation Heatmap', fontsize=14)

plt.tight_layout()
plt.show()